In [10]:
import numpy as np
from scipy.linalg import solve_banded
from scipy.interpolate import interp1d
from scipy.stats import norm # 提前引入 norm for BS formula

def crank_nicolson_american_put_log_price_fixed(S0, K, T, r, sigma, M, N, S_min=0.01, S_max=50, S_max_factor=2):
    """
    使用 Crank-Nicolson 有限差分方法计算美式看跌期权价格。
    采用对数资产价格 (x = ln(S)) 坐标以保证数值稳定性。
    已修复三对角乘法中的广播错误。
    """

    # --- 1. 网格设置和步长 (x = ln(S) 坐标) ---
    dt = T / M               # 时间步长
    
    # 设定 X 坐标范围 [X_min, X_max]
    X_max = np.log(K * S_max_factor)
    X_min = np.log(S_min) 
    
    dX = (X_max - X_min) / N # 空间步长
    
    # 资产价格和时间网格
    X = np.linspace(X_min, X_max, N + 1)
    S = np.exp(X) # 实际价格网格
    
    V = np.zeros((N + 1, M + 1))

    # --- 2. 修正后的常数系数 (在 X 坐标下) ---
    # 常数系数
    a = 0.25 * dt * (sigma**2 / dX**2 - (r - 0.5 * sigma**2) / dX)
    b = 0.5 * dt * (sigma**2 / dX**2 + r)
    c = 0.25 * dt * (sigma**2 / dX**2 + (r - 0.5 * sigma**2) / dX)
    
    # --- 3. 构造三对角矩阵 M_L 的带状存储 ---
    
    # M_L: 隐式部分 (左侧矩阵)
    # M_L = diag(1 + b) + diag(-a, k=-1) + diag(-c, k=1)
    ML_bands = np.zeros((3, N - 1))
    ML_bands[0, 1:] = -c     # 上对角线 (i+1)
    ML_bands[1, :] = 1 + b   # 主对角线 (i)
    ML_bands[2, :-1] = -a   # 下对角线 (i-1)
    
    # --- 4. 边界和终止条件 ---
    V[:, M] = np.maximum(0, K - S) # 终止条件
    time_points = np.linspace(0, T, M + 1)
    V[0, :] = K * np.exp(-r * (T - time_points)) # S=S_min 边界
    V[N, :] = 0 # S=S_max 边界

    # --- 5. 逆向求解 (Backward Induction) ---
    for j in range(M - 1, -1, -1):
        V_next = V[1:N, j + 1] # 长度 N-1
        
        # 5.1. M_R * V^{j+1} 的计算 (N-1 维向量)
        MR_V_next = np.zeros(N - 1)
        
        # 主对角线: (1 - b) * V_{i}^{j+1}
        MR_V_next += (1 - b) * V_next 
        
        # 上对角线: c * V_{i+1}^{j+1} (作用于 MR_V_next[0] 到 MR_V_next[N-3])
        MR_V_next[:-1] += c * V_next[1:] 
        
        # 下对角线: a * V_{i-1}^{j+1} (作用于 MR_V_next[1] 到 MR_V_next[N-2])
        MR_V_next[1:] += a * V_next[:-1] 

        # 5.2. 构造右侧向量 R_Vector (R_Vector = M_R * V^{j+1} + Boundary_Terms_Explicit + Boundary_Terms_Implicit)
        R_Vector = MR_V_next.copy()
        
        # 显式边界影响项 (V^{j+1})
        # R_1 受到 a * V_{0}^{j+1} 的影响
        R_Vector[0] += a * V[0, j + 1]             
        # R_{N-1} 受到 c * V_{N}^{j+1} 的影响
        R_Vector[-1] += c * V[N, j + 1]            

        # 隐式边界影响项 (V^{j})
        # R_1 受到 -a * V_{0}^{j} 的影响，因此 R_Vector[0] += -(-a * V[0, j])
        R_Vector[0] += a * V[0, j]                  
        # R_{N-1} 受到 -c * V_{N}^{j} 的影响，因此 R_Vector[-1] += -(-c * V[N, j])
        R_Vector[-1] += c * V[N, j]                 
        
        # 5.3. 求解线性方程组 M_L * V^{j} = R_Vector
        V_interior_Continuation = solve_banded((1, 1), ML_bands, R_Vector)
        
        # 5.4. 早期行权检查 (投影)
        Intrinsic_Value = np.maximum(0, K - S[1:N])
        V[1:N, j] = np.maximum(Intrinsic_Value, V_interior_Continuation)

    # --- 6. 结果插值 ---
    interpolator = interp1d(S, V[:, 0])
    
    if S0 < S_min or S0 > S_max:
         # S0 5.0, S_min 0.01, S_max 12.0. S0 在范围内。
         # 这是一个安全检查，实际运行中可能不会触发
        return "S0不在网格范围内，请调整 S_min 或 S_max_factor"

    price = interpolator(S0)
    
    return price

# --- 使用示例 ---
S0 = 5.0
K = 6.0
T = 1.0
r = 0.05
sigma = 0.3
M = 10  # 时间步数
N = 10  # 空间步数 (价格步数)

# 计算美式看跌期权价格
P_American = crank_nicolson_american_put_log_price_fixed(S0, K, T, r, sigma, M, N)
print(P_American)
# 欧洲期权价格计算 (使用解析 Black-Scholes 公式)
def black_scholes_put(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    P_European = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    return P_European

P_European = black_scholes_put(S0, K, T, r, sigma)

# 计算 EEP
EEP = P_American - P_European

print(f"CN-FDM (M={M}, N={N}) 美式看跌期权价格: {P_American:.7f}")
print(f"Black-Scholes 欧洲看跌期权价格: {P_European:.7f}")
print(f"早期行权溢价 (EEP): {EEP:.7f}")

1.1517008815937515
CN-FDM (M=10, N=10) 美式看跌期权价格: 1.1517009
Black-Scholes 欧洲看跌期权价格: 1.0525764
早期行权溢价 (EEP): 0.0991245
